<a href="https://colab.research.google.com/github/Vihang26/dl_t2i/blob/main/stable_diffusion_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install transformers diffusers lpips accelerate

In [2]:
# !pip install ipywidgets

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import torch
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, LMSDiscreteScheduler
from tqdm.auto import tqdm
from torch import autocast
from PIL import Image
from matplotlib import pyplot as plt
import numpy
from torchvision import transforms as tfms

# For video display:
from IPython.display import HTML
from base64 import b64encode

# Set device
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# Load the autoencoder model which will be used to decode the latents into image space.
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae", use_auth_token=True)

# Load the tokenizer and text encoder to tokenize and encode the text.
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet", use_auth_token=True)

# The noise scheduler
scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)

# To the GPU we go!
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device)

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.23.layer_norm1.weight', 'vision_model.encoder.layers.22.layer_norm2.bias', 'vision_model.encoder.layers.0.self_attn.out_proj.weight', 'vision_model.encoder.layers.2.layer_norm1.weight', 'vision_model.encoder.layers.5.layer_norm1.bias', 'vision_model.encoder.layers.11.self_attn.q_proj.weight', 'vision_model.encoder.layers.9.layer_norm2.bias', 'vision_model.encoder.layers.3.mlp.fc2.weight', 'vision_model.encoder.layers.19.layer_norm2.weight', 'vision_model.encoder.layers.2.layer_norm2.weight', 'vision_model.encoder.layers.10.mlp.fc2.weight', 'vision_model.encoder.layers.21.self_attn.q_proj.bias', 'vision_model.encoder.layers.17.self_attn.v_proj.weight', 'vision_model.encoder.layers.14.self_attn.q_proj.bias', 'vision_model.encoder.layers.3.layer_norm1.weight', 'vision_model.encoder.layers.8.layer_norm1.weight', 'vision_model.encoder.layers.22.m

In [8]:
# Some settings
# prompt = ["a purple jupiter next to truck."]
# prompt = ["a green okra to the left of a yellow glacier"]
def stable_diff_exec(prompt,prompt_num):
    height = 512                        # default height of Stable Diffusion
    width = 768                        # default width of Stable Diffusion
    num_inference_steps = 50            # Number of denoising steps
    guidance_scale = 7.5                # Scale for classifier-free guidance
    generator = torch.manual_seed(prompt_num)   # Seed generator to create the inital latent noise
    batch_size = 1
    
    # Prep text
    text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
    with torch.no_grad():
        text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
    max_length = text_input.input_ids.shape[-1]
    uncond_input = tokenizer(
        [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
    )
    with torch.no_grad():
        uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]
    text_embeddings = torch.cat([uncond_embeddings, text_embeddings])
    
    # Prep Scheduler
    def set_timesteps(scheduler, num_inference_steps):
        scheduler.set_timesteps(num_inference_steps)
        scheduler.timesteps = scheduler.timesteps.to(torch.float32) # minor fix to ensure MPS compatibility, fixed in diffusers PR 3925
    
    set_timesteps(scheduler,num_inference_steps)
    
    # Prep latents
    latents = torch.randn(
      (batch_size, unet.in_channels, height // 8, width // 8),
      generator=generator,
    )
    latents = latents.to(torch_device)
    latents = latents * scheduler.init_noise_sigma # Scaling (previous versions did latents = latents * self.scheduler.sigmas[0]
    
    # Loop
    with autocast("cuda"):  # will fallback to CPU if no CUDA; no autocast for MPS
        for i, t in tqdm(enumerate(scheduler.timesteps), total=len(scheduler.timesteps)):
            # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
            latent_model_input = torch.cat([latents] * 2)
            sigma = scheduler.sigmas[i]
            # Scale the latents (preconditioning):
            # latent_model_input = latent_model_input / ((sigma**2 + 1) ** 0.5) # Diffusers 0.3 and below
            latent_model_input = scheduler.scale_model_input(latent_model_input, t)
    
            # predict the noise residual
            with torch.no_grad():
                noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample
    
            # perform guidance
            noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
            noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)
    
            # compute the previous noisy sample x_t -> x_t-1
            # latents = scheduler.step(noise_pred, i, latents)["prev_sample"] # Diffusers 0.3 and below
            latents = scheduler.step(noise_pred, t, latents).prev_sample
    
    # scale and decode the image latents with vae
    latents = 1 / 0.18215 * latents
    with torch.no_grad():
        image = vae.decode(latents).sample
    
    # Display
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    save_img = pil_images[0].save(f"./level1_img/{prompt}_{str(prompt_num).zfill(6)}.png")


In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("level1.txt", header=None)
lvl_1_prompts = df[0].sample(n=500,random_state = 42)

In [7]:
lvl_1_prompts

423         a short envelope
548            a pink desert
959          three ice cream
587    a yellow paintbrushes
827           a yellow whisk
               ...          
730           a purple table
567         a yellow factory
740       a purple mountains
491    an orange bell pepper
588          a pink envelope
Name: 0, Length: 500, dtype: object

In [12]:
with open('level_1_gen.txt', 'w') as f:
    for i in lvl_1_prompts:
        f.write(i)
        f.write('\n')

In [ ]:
for i,prompt in enumerate(lvl_1_prompts):
    stable_diff_exec(prompt,i)

In [46]:
df = pd.read_csv("level2.txt", header=None)
lvl_2_prompts = df[0].sample(n=500,random_state = 11)

In [47]:
lvl_2_prompts

389                two peach to the right of orange
121           an orange forest far away from grater
522           two Meryl Streep in front of scissors
309                         a green bicycle on fall
647    an orange Sydney Opera House in between kiwi
                           ...                     
479                 a big butterfly under ambulance
342                     a little bench after school
257                 a yellow rocket next to cupcake
519                      two peach on top of pencil
659             an orange forest beside post office
Name: 0, Length: 500, dtype: object

In [48]:
with open('level_2_gen.txt', 'w') as f:
    for i in lvl_2_prompts:
        f.write(i)
        f.write('\n')

In [23]:
df = pd.read_csv("level3.txt", header=None)
lvl_3_prompts = df[0].sample(n=500,random_state = 101)

In [24]:
lvl_3_prompts

240       a square Burj Khalifa on top of one doctor
304                a yellow okra near a narrow Venus
411       a green school on top of a green ambulance
336    a purple post office behind an orange lettuce
271     a narrow cricket match beside a big mushroom
                           ...                      
217                  a blue train with a short chair
513    a rectangular waffle after a purple butterfly
305               a green school after a narrow kiwi
590                 a yellow okra after a round bank
459                  a pink plains beside one doctor
Name: 0, Length: 500, dtype: object

In [25]:
with open('level_3_gen.txt', 'w') as f:
    for i in lvl_3_prompts:
        f.write(i)
        f.write('\n')